In [1]:
import os
import shutil
 
original_dataset_dir = './dataset'   
classes_list = os.listdir(original_dataset_dir) 
 
base_dir = './splitted' 
os.mkdir(base_dir)
 
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list[0:4]:     
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

In [2]:
import math
 
for cls in classes_list[0:4]:
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)
    
    train_fnames = fnames[:train_size]
    print("Train size(",cls,"): ", len(train_fnames))
    for fname in train_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(train_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Apple___Apple_scab ):  378
Validation size( Apple___Apple_scab ):  126
Test size( Apple___Apple_scab ):  126
Train size( Apple___Black_rot ):  372
Validation size( Apple___Black_rot ):  124
Test size( Apple___Black_rot ):  124
Train size( Apple___Cedar_apple_rust ):  165
Validation size( Apple___Cedar_apple_rust ):  55
Test size( Apple___Cedar_apple_rust ):  55
Train size( Apple___healthy ):  987
Validation size( Apple___healthy ):  329
Test size( Apple___healthy ):  329


In [3]:
import matplotlib.pyplot as plt

In [4]:
import torch
import os
 
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 

In [5]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
 
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()]) 
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base) 
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)

In [6]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [7]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module): 
  
    def __init__(self): 
    
        super(Net, self).__init__() 

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) 
        self.pool = nn.MaxPool2d(2,2)  
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)  

        self.fc1 = nn.Linear(4096, 512) 
        self.fc2 = nn.Linear(512, 33) 
    
    def forward(self, x):  
    
        x = self.conv1(x)
        x = F.relu(x)  
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training) 

        x = self.conv2(x)
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x) 
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1, 4096)  
        x = self.fc1(x) 
        x = F.relu(x) 
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x) 

        return F.log_softmax(x, dim=1)  
        

model_base = Net().to(DEVICE)  
optimizer = optim.Adam(model_base.parameters(), lr=0.001) 

In [8]:
def train(model, train_loader, optimizer):
    model.train()  
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) 
        optimizer.zero_grad() 
        output = model(data)  
        loss = F.cross_entropy(output, target) 
        loss.backward()  
        optimizer.step()  

In [9]:
def predict():
    data, target = next(iter(val_loader))
    outputs= model_base(data)
    _, predicted = torch.max(outputs, 1)
    return predicted, target

In [20]:
def evaluate(model, test_loader):
    model.eval()  
    test_loss = 0 
    correct = 0   
    
    with torch.no_grad(): 
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)  
            output = model(data) 
            
            test_loss += F.cross_entropy(output,target, reduction='sum').item() 
 
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item() 
   
    test_loss /= len(test_loader.dataset) 
    test_accuracy = 100. * correct / len(test_loader.dataset) 
    return test_loss, test_accuracy  

In [14]:
predicted_result, target = predict()

In [15]:
for i in range(len(predicted_result)):
    print("{}:predicted={}, target={}".format(i, predicted_result[i], target[i]))

0:predicted=25, target=3
1:predicted=25, target=3
2:predicted=8, target=3
3:predicted=20, target=1
4:predicted=0, target=3
5:predicted=8, target=2
6:predicted=28, target=0
7:predicted=25, target=1
8:predicted=0, target=0
9:predicted=0, target=2
10:predicted=25, target=1
11:predicted=25, target=1
12:predicted=13, target=3
13:predicted=13, target=1
14:predicted=0, target=1
15:predicted=13, target=3
16:predicted=0, target=0
17:predicted=23, target=3
18:predicted=8, target=3
19:predicted=28, target=1
20:predicted=0, target=3
21:predicted=25, target=1
22:predicted=13, target=0
23:predicted=13, target=2
24:predicted=19, target=3
25:predicted=13, target=0
26:predicted=0, target=3
27:predicted=8, target=1
28:predicted=8, target=3
29:predicted=28, target=0
30:predicted=25, target=0
31:predicted=20, target=1
32:predicted=28, target=0
33:predicted=0, target=3
34:predicted=8, target=3
35:predicted=13, target=3
36:predicted=16, target=3
37:predicted=4, target=2
38:predicted=25, target=3
39:predicte

In [22]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader) 
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train Loss: 0.2779, Accuracy: 88.54%
val Loss: 0.3762, Accuracy: 85.96%
Completed in 0m 34s
-------------- epoch 2 ----------------
train Loss: 0.1239, Accuracy: 96.16%
val Loss: 0.2103, Accuracy: 93.06%
Completed in 0m 34s
-------------- epoch 3 ----------------
train Loss: 0.1189, Accuracy: 96.48%
val Loss: 0.2115, Accuracy: 92.59%
Completed in 0m 30s
-------------- epoch 4 ----------------
train Loss: 0.1883, Accuracy: 92.80%
val Loss: 0.2659, Accuracy: 91.48%
Completed in 0m 30s
-------------- epoch 5 ----------------
train Loss: 0.1654, Accuracy: 94.79%
val Loss: 0.2586, Accuracy: 92.59%
Completed in 0m 30s
-------------- epoch 6 ----------------
train Loss: 0.1319, Accuracy: 95.79%
val Loss: 0.2257, Accuracy: 93.06%
Completed in 0m 31s
-------------- epoch 7 ----------------
train Loss: 0.1243, Accuracy: 96.16%
val Loss: 0.2169, Accuracy: 92.43%
Completed in 0m 30s
-------------- epoch 8 ----------------
train Loss: 0.1105, Accuracy: 96.42%